## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_to_load = ("../Weather_Database/WeatherPy_Database.csv")

city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Longyearbyen,SJ,78.2186,15.6401,42.64,70,75,10.36,broken clouds
1,1,Clyde River,CA,70.4692,-68.5914,35.83,80,20,36.82,few clouds
2,2,Busselton,AU,-33.6500,115.3333,55.83,72,97,22.77,overcast clouds
3,3,Arroyo,ES,41.6096,-4.7969,87.84,30,75,13.80,thunderstorm
4,4,Svetlogorsk,BY,52.6333,29.7333,57.40,87,100,3.85,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Arroyo,ES,41.6096,-4.7969,87.84,30,75,13.80,thunderstorm
6,6,Ilhabela,BR,-23.7781,-45.3581,76.91,57,2,4.00,clear sky
7,7,Vanimo,PG,-2.6741,141.3028,76.51,88,27,3.71,scattered clouds
22,22,Butaritari,KI,3.0707,172.7902,82.15,77,11,14.83,light rain
25,25,Alofi,NU,-19.0595,-169.9187,75.09,88,75,12.66,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df  

preferred_cities_clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Arroyo,ES,41.6096,-4.7969,87.84,30,75,13.80,thunderstorm
6,6,Ilhabela,BR,-23.7781,-45.3581,76.91,57,2,4.00,clear sky
7,7,Vanimo,PG,-2.6741,141.3028,76.51,88,27,3.71,scattered clouds
22,22,Butaritari,KI,3.0707,172.7902,82.15,77,11,14.83,light rain
25,25,Alofi,NU,-19.0595,-169.9187,75.09,88,75,12.66,broken clouds
...,...,...,...,...,...,...,...,...,...,...
692,692,Mahad,IN,18.0833,73.4167,81.52,82,100,2.71,overcast clouds
693,693,Maragogi,BR,-9.0122,-35.2225,79.43,75,76,1.43,broken clouds
694,694,Puerto El Triunfo,SV,13.2833,-88.5500,83.23,70,86,9.73,overcast clouds
705,705,Axim,GH,4.8699,-2.2405,75.78,87,100,9.04,light rain


In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Arroyo,ES,87.84,thunderstorm,41.6096,-4.7969,
6,Ilhabela,BR,76.91,clear sky,-23.7781,-45.3581,
7,Vanimo,PG,76.51,scattered clouds,-2.6741,141.3028,
22,Butaritari,KI,82.15,light rain,3.0707,172.7902,
25,Alofi,NU,75.09,broken clouds,-19.0595,-169.9187,
...,...,...,...,...,...,...,...
692,Mahad,IN,81.52,overcast clouds,18.0833,73.4167,
693,Maragogi,BR,79.43,broken clouds,-9.0122,-35.2225,
694,Puerto El Triunfo,SV,83.23,overcast clouds,13.2833,-88.5500,
705,Axim,GH,75.78,light rain,4.8699,-2.2405,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Arroyo,ES,87.84,thunderstorm,41.6096,-4.7969,Hotel El Jardín de la Abadía
6,Ilhabela,BR,76.91,clear sky,-23.7781,-45.3581,Ilha Flat Hotel
7,Vanimo,PG,76.51,scattered clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
22,Butaritari,KI,82.15,light rain,3.0707,172.7902,Isles Sunset Lodge
25,Alofi,NU,75.09,broken clouds,-19.0595,-169.9187,Taloa Heights
...,...,...,...,...,...,...,...
692,Mahad,IN,81.52,overcast clouds,18.0833,73.4167,Hotel Saitej
693,Maragogi,BR,79.43,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
694,Puerto El Triunfo,SV,83.23,overcast clouds,13.2833,-88.5500,Hotel Jardin
705,Axim,GH,75.78,light rain,4.8699,-2.2405,Axim Beach Resort And Spa


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))